## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Qaanaaq,77.4840,-69.3632,29.48,87,100,11.18,GL,2021-10-10 17:12:52
1,1,Nome,64.5011,-165.4064,26.67,93,40,5.75,US,2021-10-10 17:12:52
2,2,Inirida,3.8653,-67.9239,90.90,54,97,2.55,CO,2021-10-10 17:12:53
3,3,Albany,42.6001,-73.9662,67.17,75,91,3.00,US,2021-10-10 17:10:28
4,4,Yangmei,24.1023,112.5618,62.64,96,100,6.64,CN,2021-10-10 17:12:53


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,Albany,42.6001,-73.9662,67.17,75,91,3.00,US,2021-10-10 17:10:28
5,5,Dicabisagan,17.0818,122.4157,78.58,87,99,6.87,PH,2021-10-10 17:12:53
8,8,Indianola,41.3581,-93.5574,73.31,68,100,7.81,US,2021-10-10 17:12:54
10,10,Ponta Do Sol,32.6667,-17.1000,76.71,71,13,2.95,PT,2021-10-10 17:12:55
15,15,Omboue,-1.5746,9.2618,77.85,82,97,7.31,GA,2021-10-10 17:12:56
16,16,Vaini,-21.2000,-175.2000,69.96,100,75,2.30,TO,2021-10-10 17:11:04
17,17,Dunkwa,5.9656,-1.7800,77.81,94,76,4.09,GH,2021-10-10 17:12:56
18,18,Rikitea,-23.1203,-134.9692,74.62,81,100,19.13,PF,2021-10-10 17:12:57
19,19,Kapaa,22.0752,-159.3190,76.12,77,75,5.01,US,2021-10-10 17:12:57
22,22,Sola,-13.8833,167.5500,79.38,79,99,18.12,VU,2021-10-10 17:12:58


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID       230
City          230
Lat           230
Lng           230
Max Temp      230
Humidity      230
Cloudiness    230
Wind Speed    230
Country       229
Date          230
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Albany,US,67.17,42.6001,-73.9662,
5,Dicabisagan,PH,78.58,17.0818,122.4157,
8,Indianola,US,73.31,41.3581,-93.5574,
10,Ponta Do Sol,PT,76.71,32.6667,-17.1000,
15,Omboue,GA,77.85,-1.5746,9.2618,
16,Vaini,TO,69.96,-21.2000,-175.2000,
17,Dunkwa,GH,77.81,5.9656,-1.7800,
18,Rikitea,PF,74.62,-23.1203,-134.9692,
19,Kapaa,US,76.12,22.0752,-159.3190,
22,Sola,VU,79.38,-13.8833,167.5500,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")
    

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Albany,US,67.17,42.6001,-73.9662,
5,Dicabisagan,PH,78.58,17.0818,122.4157,
8,Indianola,US,73.31,41.3581,-93.5574,
10,Ponta Do Sol,PT,76.71,32.6667,-17.1000,
15,Omboue,GA,77.85,-1.5746,9.2618,
...,...,...,...,...,...,...
689,Axim,GH,78.15,4.8699,-2.2405,
690,Yenagoa,NG,79.70,4.9247,6.2642,
692,Hargeysa,SO,75.04,9.5600,44.0650,
693,Dawei,MM,76.69,14.0833,98.2000,


In [18]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
